In [1]:
import opensmile

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.LowLevelDescriptors
)
y = smile.process_file('./v1/v1_01.mp3')
list_of_features = ['mfcc1_sma3','mfcc2_sma3','mfcc3_sma3','mfcc4_sma3']
y.head()

Loudness_sma3  \
file           start                  end                                     
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000       0.001034   
               0 days 00:00:00.010000 0 days 00:00:00.030000       0.001034   
               0 days 00:00:00.020000 0 days 00:00:00.040000       0.003848   
               0 days 00:00:00.030000 0 days 00:00:00.050000       0.018856   
               0 days 00:00:00.040000 0 days 00:00:00.060000       0.102363   

                                                              alphaRatio_sma3  \
file           start                  end                                       
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000         0.000000   
               0 days 00:00:00.010000 0 days 00:00:00.030000         0.000000   
               0 days 00:00:00.020000 0 days 00:00:00.040000         2.393254   
               0 days 00:00:00.030000 0 days 00:00:00.050000         0.810536   
               0 days 00:00:00.040000 0 days 00:00:00.060000        -0.503224   

                                                              hammarbergIndex_sma3  \
file           start                  end                                            
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000              0.000000   
               0 days 00:00:00.010000 0 days 00:00:00.030000              0.000000   
               0 days 00:00:00.020000 0 days 00:00:00.040000             -0.721510   
               0 days 00:00:00.030000 0 days 00:00:00.050000              2.534581   
               0 days 00:00:00.040000 0 days 00:00:00.060000              5.461428   

                                                              slope0-500_sma3  \
file           start                  end                                       
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000         0.164599   
               0 days 00:00:00.010000 0 days 00:00:00.030000         0.164599   
               0 days 00:00:00.020000 0 days 00:00:00.040000         0.150489   
               0 days 00:00:00.030000 0 days 00:00:00.050000         0.120536   
               0 days 00:00:00.040000 0 days 00:00:00.060000         0.059767   

                                                              slope500-1500_sma3  \
file           start                  end                                          
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000            0.028635   
               0 days 00:00:00.010000 0 days 00:00:00.030000            0.028635   
               0 days 00:00:00.020000 0 days 00:00:00.040000            0.022688   
               0 days 00:00:00.030000 0 days 00:00:00.050000            0.013211   
               0 days 00:00:00.040000 0 days 00:00:00.060000            0.003382   

                                                              spectralFlux_sma3  \
file           start                  end                                         
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000           0.000000   
               0 days 00:00:00.010000 0 days 00:00:00.030000           0.000000   
               0 days 00:00:00.020000 0 days 00:00:00.040000           0.000134   
               0 days 00:00:00.030000 0 days 00:00:00.050000           0.002279   
               0 days 00:00:00.040000 0 days 00:00:00.060000           0.029390   

                                                              mfcc1_sma3  \
file           start                  end                                  
./v1/v1_01.mp3 0 days 00:00:00        0 days 00:00:00.020000    0.000000   
               0 days 00:00:00.010000 0 days 00:00:00.030000    0.000000   
               0 days 00:00:00.020000 0 days 00:00:00.040000   -2.116095   
               0 days 00:00:00.030000 0 days 00:00:00.050000    0.038239   
               0 days 00:00:00.040000 0 days 00:00:00.060000    1.759300   

                                                              mfc

In [2]:
import plotly.express as px

DURATION = 0.02
time_list = []
for i in range(y.shape[0]):
    time_list.append(f"00:00:0{i/100}")

y['Time'] = time_list
px.line(y,y=list_of_features,x='Time',height=600,width=1000)